# Test

In [1]:
import molsysmt as msm

In [2]:
import openmm as mm
from openmm import app
from openmm import unit

In [3]:
#molsys = msm.convert('4POC')
#molsys = msm.extract(molsys, selection='molecule_type=="protein"')
#molsys = msm.build.add_missing_hydrogens(molsys)
#molsys = msm.build.solvate(molsys)

In [4]:
molsys = msm.convert('initial.msmpk')

In [5]:
topology = msm.convert(molsys, 'openmm.Topology')
coordinates = msm.get(molsys, coordinates=True)
positions = msm.pyunitwizard.convert(coordinates[0], to_form='openmm.unit')

In [6]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [7]:
system = forcefield.createSystem(topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [8]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

In [9]:
platform = mm.Platform.getPlatformByName("CUDA")

In [10]:
simulation = app.Simulation(topology, system, integrator, platform)

In [11]:
simulation.context.setPositions(positions)

In [12]:
#simulation.minimizeEnergy()
#msm.convert(simulation, 'initial.msmpk')

In [13]:
integration_time_step = 2.0*unit.femtoseconds
total_time = 0.1 * unit.nanoseconds
total_steps = round(total_time / integration_time_step)
interval_time = 10 * unit.picoseconds
interval_steps = round(interval_time / integration_time_step)

In [14]:
from molsysmt.thirds.openmm.reporters import MSMH5Reporter, TQDMReporter

In [15]:
print(interval_steps, total_steps)

5000 50000


In [16]:
tqdm_reporter = TQDMReporter(100, total_steps)
simulation.reporters.append(tqdm_reporter)

In [17]:
msmh5_reporter = MSMH5Reporter('traj.msmh5', interval_steps, steps=total_steps, selection='all',
            topology=topology, time=True, box=True, coordinates=True, velocities=False,
            potentialEnergy=True, kineticEnergy=True, temperature=True,
            includeInitialContext=True, constantReportInterval=True,
            constantStepSize=True, constantBox=True,
            auto_close=True)
simulation.reporters.append(msmh5_reporter)

In [18]:
simulation.step(total_steps)

  0%|          | 0/50000 [00:00<?, ?it/s]


Potential energy: -533115.42 kJ/mol ± 12653.80 kJ/mol
Temperature: 296.25 K ± 23.48 K

Execution time: 0 days, 0 hours, 0 minutes, and 57.2 seconds (151.043 ns/day).

